In [53]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()


from skimage import data, io, transform
from skimage.color import rgb2gray
import matplotlib.pyplot as plt
import numpy as np
import os

In [54]:
with tf.compat.v1.Session() as sess:
    
    #Build a Graph
    a = tf.constant([1,2,3,4,5])
    b = tf.constant([6,7,8,9,10])

    result = tf.multiply(a,b)

    print(sess.run(result))

[ 6 14 24 36 50]


In [55]:
def load_data(data_directory):
    directories = [d for d in os.listdir(data_directory) 
                   if os.path.isdir((os.path.join(data_directory,d)))] 
    
    #print(os.listdir(data_directory))
    labels,images = [],[]
    
    for d in directories:
        label_directory = os.path.join(data_directory,d)
        
        file_names = [os.path.join(label_directory,f) 
                      for f in os.listdir(label_directory)
                      if f.endswith('.ppm')]
        
        for f in file_names:
            images.append(skimage.io.imread(f))
            labels.append(int(d))
    
    return images,labels

In [52]:
ROOT_PATH = '/Users/ModouNiang/Desktop/Neural-Network/'

#Creating Directiories containing training and testing data
train_data_directory = os.path.join(ROOT_PATH,'Training')
test_data_directory = os.path.join(ROOT_PATH,'Testing')

#Training data
images,labels = load_data(train_data_directory)

images = np.array(images)
#labels = np.array(labels)

print(len(set(labels)))

NameError: name 'skimage' is not defined

## Data Exploration

In [ ]:
print('Image Dimesions: ',images.ndim)
print('Number of images: ',images.size)
#print(images[0])

In [ ]:
print('Label Dimensions: ',labels.ndim)
print('Number of Labels: ',len(set(labels)))

In [ ]:
plt.hist(labels,62)
plt.title('Distribution of Traffic Sign Labels')
plt.show()

In [ ]:
#Choosing random images and displaying them within a subplot
traffic_signs = [300,2250,3650,4000]

#Creating subplots to display the random images
for i in range(len(traffic_signs)):
    plt.subplot(1,4,i+1)
    plt.axis('off')
    plt.imshow(images[traffic_signs[i]])
    plt.subplots_adjust(wspace=0.5)
    
    print('Shape: {}, Minimum: {}, Maximum: {}'.format(images[traffic_signs[i]].shape,
                                                       images[traffic_signs[i]].min(),
                                                       images[traffic_signs[i]].max()))

plt.show()

## Displaying Unique Labels

In [ ]:
unique_labels = set(labels)
unique_labels
plt.figure(figsize=(15,15))

counter = 1
for label in unique_labels:
    
    image = images[labels.index(label)]
    
    #Define 64 subplots
    plt.subplot(8, 8, i)

    plt.axis('off')
    
    plt.title('Label {} {}'.format(label,labels.count(label)))
    
    counter += 1
    
    plt.imshow(image)
    
plt.show()

In [ ]:
#Rescaling images to 28x28 images
images28 = [transform.resize(image,(28,28)) for image in images]
images28 = np.array(images28)

#Choosing random images and displaying them within a subplot
traffic_signs = [300,2250,3650,4000]

#Creating subplots to display the random images
for i in range(len(traffic_signs)):
    plt.subplot(1,4,i+1)
    plt.axis('off')
    plt.imshow(images28[traffic_signs[i]])
    plt.subplots_adjust(wspace=0.5)
    
    print('Shape: {}, Minimum: {}, Maximum: {}'.format(images28[traffic_signs[i]].shape,
                                                       images28[traffic_signs[i]].min(),
                                                       images28[traffic_signs[i]].max()))

plt.show()

In [ ]:
#Converting images to grayscale
images28 = np.array(images28)
images28 = rgb2gray(images28)

plt.imshow(images28[0])

#Choosing random images and displaying them within a subplot
traffic_signs = [300,2250,3650,4000]

#Creating subplots to display the random images
for i in range(len(traffic_signs)):
    plt.subplot(1,4,i+1)
    plt.axis('off')
    plt.imshow(images28[traffic_signs[i]], cmap='gray')
    plt.subplots_adjust(wspace=0.5)

plt.show()

## Constructing the Neural Network

In [ ]:
#Defining placeholders for inputs and labels to be later used during the session
x = tf.placeholder(dtype = float32, shape = [None,28,28])
y = tf.placeholder(dtype = float32, shape = [None])

#Flattening the input data
flattened_images = tf.contrib.layers.flatten(x)

#Constructing a fully connected layer that generates logits 
logits = tf.contrib.layers.fully_connected(flattened_images,62,tf.nn.relu)

#Defining loss function
loss_func = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels = y,
                                                                         logits = logits))

#Define optimizer
train_op = tf.train.AdamOptimizer(learning_rate = .001).minimize(loss)

#Converting logits to label indicies
pred = tf.argmax(logits,1)

#Creating an accuracy metrix
accuracy = tf.reduce_mean(tf.cast(correct_pred,tf.float32))